In [1]:
%load_ext sql
import pandas as pd
import sqlite3

## Create the Database

In [2]:
%sql sqlite:///CourseData.db 
conn = sqlite3.connect('CourseData.db')

Add all tables to Database and connect them

In [3]:
%%sql
DROP TABLE IF EXISTS CATALOGCOURSE;
CREATE TABLE CATALOGCOURSE (
    CourseID INTEGER PRIMARY KEY,
    catalog_id TEXT NOT NULL,
    course_title TEXT NOT NULL,
    credits INTEGER,
    prereqs TEXT,
    coreqs TEXT,
    fees INTEGER,
    attributes TEXT,
    description TEXT,
    program_code TEXT NOT NULL
);
CREATE INDEX ix_CourseID_alt_key on CATALOGCOURSE(CourseID);

DROP TABLE IF EXISTS TERMS;
CREATE TABLE TERMS (
    term_code INTEGER PRIMARY KEY,
    term TEXT NOT NULL,
    FOREIGN KEY (term_code) REFERENCES CATALOGCOURSE (term_code)
);
CREATE INDEX ix_term_code_alt_key on TERMS(term);

DROP TABLE IF EXISTS SECTION;
CREATE TABLE SECTION (
    section_id INTEGER PRIMARY KEY,
    crn INTEGER,
    title TEXT,
    section TEXT,
    credits INTERGER,
    cap INTEGER,
    act INTEGER,
    remaining INTEGER,
    time_codes TEXT,
    catalog_id TEXT NOT NULL,
    instructor_name TEXT,
    term TEXT,
    FOREIGN KEY (catalog_id) REFERENCES CATALOGCOURSE (catalog_id)
    
);
CREATE INDEX ix_section_id_alt_key on SECTION(crn);

DROP TABLE IF EXISTS SECTIONMEETINGS;
CREATE TABLE SECTIONMEETINGS (
    section_meetings_id INTEGER PRIMARY KEY,
    crn INTEGER,
    start TEXT,
    end TEXT,
    day TEXT,
    location_code TEXT,
    section_id INTEGER,
    FOREIGN KEY (crn) REFERENCES SECTION (crn),
    FOREIGN KEY (section_id) REFERENCES SECTION (section_id)
);
CREATE INDEX ix_section_meetings_id_alt_key on SECTIONMEETINGS(section_meetings_id);

DROP TABLE IF EXISTS LOCATIONS;
CREATE TABLE LOCATIONS (
    location_code TEXT PRIMARY KEY,
    building TEXT,
    room_number INTEGER,
    FOREIGN KEY (location_code) REFERENCES SECTIONMEETINGS (location_code)
);

DROP TABLE IF EXISTS INSTRUCTORS;
CREATE TABLE INSTRUCTORS (
    instructor_id INTEGER NOT NULL PRIMARY KEY,
    instructor_name TEXT NOT NULL,
    FOREIGN KEY (instructor_id) REFERENCES SECTION (instructor_id)
);
CREATE INDEX ix_instructor_id_alt_key on INSTRUCTORS(instructor_name);

DROP TABLE IF EXISTS PROGRAMS;
CREATE TABLE PROGRAMS (
    program_code TEXT NOT NULL PRIMARY KEY,
    program_name TEXT NOT NULL,
    FOREIGN KEY (program_code) REFERENCES CATALOGCOURSE (program_code)
);

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

Import Data into the Database from Catalogs

In [4]:
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('IMPORT_CATALOGCOURSE',conn,if_exists='append',index=False)

In [5]:
terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_SECTION',conn,if_exists='append',index=False) 
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('IMPORT_SECTIONMEETINGS',conn,if_exists='append',index=False)

In [6]:
%%sql
-- Record Counts for Catalog Courses
SELECT 
    (SELECT Count(*) FROM IMPORT_CATALOGCOURSE) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_CATALOGCOURSE)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


In [7]:
%%sql
-- Record Counts for Sections
SELECT 
    (SELECT Count(*) FROM IMPORT_SECTION) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_SECTION)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
15937,15937


In [8]:
%%sql
-- Record Counts for Section Meetings
SELECT 
    (SELECT Count(*) FROM IMPORT_SECTIONMEETINGS) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM IMPORT_SECTIONMEETINGS)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


## Populating Tables

In [9]:
%%sql
DELETE FROM INSTRUCTORS;

INSERT INTO INSTRUCTORS (instructor_name)
SELECT DISTINCT primary_instructor
FROM IMPORT_SECTION
WHERE primary_instructor <> 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [10]:
%%sql

DELETE FROM PROGRAMS;

INSERT INTO PROGRAMS (program_code,program_name)
SELECT DISTINCT program_code,program_name 
FROM IMPORT_CATALOGCOURSE
ORDER BY program_code;

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [11]:
%%sql

DELETE FROM TERMS;

INSERT INTO TERMS (term)
SELECT DISTINCT term 
FROM IMPORT_SECTIONMEETINGS;

 * sqlite:///CourseData.db
0 rows affected.
19 rows affected.


[]

In [12]:
%%sql 
DELETE FROM CATALOGCOURSE;

INSERT INTO CATALOGCOURSE (catalog_id, course_title, credits, prereqs, coreqs, fees, attributes, description, program_code)
SELECT catalog_id,course_title,credits,prereqs,coreqs,fees,attributes,description,program_code 
FROM IMPORT_CATALOGCOURSE;

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

In [13]:
%%sql 

DELETE FROM SECTION;

INSERT INTO SECTION (crn, title, section, credits, cap, act, remaining, time_codes, catalog_id, instructor_name, term)
SELECT DISTINCT crn,title,section,credits,cap,act,rem,timecodes,catalog_id,primary_instructor,term
FROM IMPORT_SECTION;

 * sqlite:///CourseData.db
0 rows affected.
15937 rows affected.


[]

In [14]:
%%sql
DELETE FROM SECTIONMEETINGS;

INSERT INTO SECTIONMEETINGS (crn, start, end, day, location_code, section_id)
SELECT DISTINCT crn,start,end,day,location, SECTION.section_id
FROM IMPORT_SECTIONMEETINGS
LEFT JOIN SECTION USING (term,crn);

 * sqlite:///CourseData.db
0 rows affected.
284847 rows affected.


[]

In [15]:
%%sql

DELETE FROM LOCATIONS;

INSERT INTO LOCATIONS (location_code)
SELECT DISTINCT location
FROM IMPORT_SECTIONMEETINGS;

 * sqlite:///CourseData.db
0 rows affected.
207 rows affected.


[]